In [1]:
print("hello NLP")

hello NLP


# V1.0 Docx汉字注音

In [3]:
import docx
from pypinyin import lazy_pinyin

# 读取Word文档
try:
    doc = docx.Document(r'D:\Learning NLP Together\UCAS-NLP-Homework\text.docx')
except FileNotFoundError:
    print("文件未找到，请检查文件路径和名称是否正确。")
    exit(1)

# 创建一个新的Word文档来存储结果
output_doc = docx.Document()
output_doc.add_paragraph("原始内容")

for para in doc.paragraphs:
    text = para.text.strip()  # 去除可能的前后空白
    pinyin_text = ' '.join([char + '(' + ''.join(lazy_pinyin(char)) + ')' for char in text if '\u4e00' <= char <= '\u9fa5'])
    
    if text:  # 只添加非空的段落
        output_doc.add_paragraph(pinyin_text)

# 将结果保存到新的文件
output_doc.save('text_with_pinyin.docx')

# V1.1 识别Doxc中的多音字

- st1:建立多音字和易错字词表
- st2:遍历文本及词表是否含有多音字或易错字
- st3:找出多音字或易错字并在后面添加注音
  - st3.1:添加注音到底添加的是哪个读音？
  - st3.2:如"大便（bian4）"、"便(pian2)宜"
  

In [32]:
# 代码示例

## 建立多音字词表
### 词表内容：多音字及读音，及相关读音的常见词组
### 读音按常见频率从上至下
duoyinzi = [
    {"便":{
        "bian4":["方便","便利","轻便","方便面"],
        "pian2":["便宜","便宜货"]
        }},
    {"好":{
        "hao3":["好人","好吃"],
        "hao4":["爱好","喜好"]
        }},
    {"打":{
        "da3":["打拳","打手"],
        "da2":["一打"]
        }}
]


In [28]:
# 便利词表
import random
## 选择随机的字典
random_entry = random.choice(duoyinzi)

## 获取字典的键（汉字）
hanzi_key = list(random_entry.keys())[0]

## 获取对应的拼音字典
pinyin_dict = random_entry[hanzi_key]

## 选择随机的拼音和对应的值
random_pinyin_key = random.choice(list(pinyin_dict.keys()))
random_value = random.choice(pinyin_dict[random_pinyin_key])

print(f"随机选择的汉字: {hanzi_key}")
print(f"对应的拼音: {random_pinyin_key}, 随机词汇: {random_value}")

随机选择的汉字: 便
对应的拼音: bian4, 随机词汇: 便利


要根据 duoyinzi 列表找出 text 文本中的多音字并给其加上注音，放在圆括号内，形成新的文本，可以按以下步骤进行：

- 分词：对文本进行分词，获得每个字。

- 查找多音字：遍历分词结果，检查每个字是否在 duoyinzi 列表中。

- 生成新文本：对于每个多音字，将其加上注音，形成新的文本。

- 以下是实现这个功能的示例代码：

示例代码

In [116]:
import re

# 多音字列表
duoyinzi = [
    {"便": {
        "bian4": ["方便", "便利", "轻便", "方便面"],
        "pian2": ["便宜", "便宜货"]
    }},
    {"好": {
        "hao3": ["好人", "好吃"],
        "hao4": ["爱好", "喜好"]
    }},
    {"打": {
        "da3": ["打拳", "打手"],
        "da2": ["一打"]
    }},
]

# 创建字典方便查询
duoyinzi_dict = {}
for item in duoyinzi:
    for word, pronunciations in item.items():
        duoyinzi_dict[word] = pronunciations

# for d in duoyinzi_dict:
#     print(d)

# print(duoyinzi_dict.items())

def add_pronunciation(sentence):
    for word, pronunciations in duoyinzi_dict.items():
        # 检查当前句子中是否包含该多音字
        if word in sentence:
            # 获取第一个拼音
            for pinyin in pronunciations.keys():
                pronunciation_str = pinyin  # 取第一个拼音
                # 替换句子中的多音字
                # print(pronunciation_str)
                sentence = sentence.replace(word, f"{word}({pronunciation_str})")
                break  # 只添加遇到的第一个拼音，其他忽略

    return sentence

# 示例输入
input_sentence = "我的爱好是打篮球"
output_sentence = add_pronunciation(input_sentence)
print(output_sentence)

我的爱好(hao3)是打(da3)篮球


# V1.2 根据词组精确注音

In [129]:
import re
import jieba

# 多音字列表
duoyinzi = [
    {"便": {
        "bian4": ["方便", "便利", "轻便", "方便面"],
        "pian2": ["便宜", "便宜货"]
    }},
    {"好": {
        "hao3": ["好人", "好吃"],
        "hao4": ["爱好", "喜好"]
    }},
    {"打": {
        "da3": ["打拳", "打手"],
        "da2": ["一打"]
    }},
]

# 创建字典方便查询
duoyinzi_dict = {}
for item in duoyinzi:
    for word, pronunciations in item.items():
        duoyinzi_dict[word] = pronunciations

def add_pronunciation(sentence):
    # jieba分词
    words = list(jieba.cut(sentence))
    output = []

    for word in words:
        found = False  # 确保是否找到多个音字
        for key, pronunciations in duoyinzi_dict.items():
            if word == key or (key in word):  # 检查匹配
                found = True
                # 检查当前词是否在对应的拼音词组中
                for pinyin_key, values in pronunciations.items():
                    if word in values:
                        pronunciation = pinyin_key  # 从对应的拼音选择
                        break
                else:
                    # 如果没有找到相应的词组，使用第一个拼音
                    pronunciation = list(pronunciations.keys())[0]
                
                output.append(f"{word}({pronunciation})")
                break  # 找到后停止继续查找

        if not found:
            output.append(word)  # 如果当前词不是多音字，则直接添加

    return ' '.join(output)

# 示例输入
# input_sentence = "我的爱好是打拳"
input_sentence = "我是好人"
# input_sentence = "这个好人真便宜"
output_sentence = add_pronunciation(input_sentence).replace(" ","")
print(output_sentence)

我是好人(hao3)


# V1.3 调整注音位置在多音字后面

In [138]:
import jieba

# 多音字列表
duoyinzi = [
    {"便": {
        "bian4": ["方便", "便利", "轻便", "方便面"],
        "pian2": ["便宜", "便宜货"]
    }},
    {"好": {
        "hao3": ["好人", "好吃"],
        "hao4": ["爱好", "喜好"]
    }},
    {"打": {
        "da3": ["打拳", "打手"],
        "da2": ["一打"]
    }},
]

# 创建字典方便查询
duoyinzi_dict = {}
for item in duoyinzi:
    for word, pronunciations in item.items():
        duoyinzi_dict[word] = pronunciations

# 定义注音函数
def add_pronunciation(sentence):
    # jieba分词
    words = list(jieba.cut(sentence))
    output = []

    for word in words:
        found = False  # 确保是否找到多个音字
        for key, pronunciations in duoyinzi_dict.items():
            if word == key or (key in word):  # 检查匹配
                found = True
                word_temp = list(word)#把词变成列表
                # 检查当前词是否在对应的拼音词组中
                for pinyin_key, values in pronunciations.items():
                    if word in values:
                        pronunciation = pinyin_key  # 从对应的拼音选择
                        break
                else:
                    # 如果没有找到相应的词组，使用第一个拼音
                    pronunciation = list(pronunciations.keys())[0]
                
                for wo in word_temp:
                    if wo == key:
                        output.append(f"{wo}({pronunciation})")
                    else:
                        output.append(f"{wo}")
                    
                break  # 找到后停止继续查找

        if not found:
            output.append(word)  # 如果当前词不是多音字，则直接添加

    return ' '.join(output)

# 示例输入
input_sentence = "我的爱好是打拳"
# input_sentence = "我是好人"
# input_sentence = "这个好人真便宜"
output_sentence = add_pronunciation(input_sentence).replace(" ","")
print(output_sentence)

我的爱好(hao4)是打(da3)拳


# V2.0 Word（Docx）文档多音字标注

In [140]:
import docx
from pypinyin import lazy_pinyin
import jieba

# 建立多音字列表
duoyinzi = [
    {"便": {
        "biàn": ["方便", "便利", "轻便", "方便面"],
        "pián": ["便宜", "便宜货"]
    }},
    {"好": {
        "hǎo": ["好人", "好吃"],
        "hào": ["爱好", "喜好"]
    }},
    {"打": {
        "dǎ": ["打拳", "打手"],
        "dá": ["一打"]
    }},
]

# 创建字典方便查询
duoyinzi_dict = {}
for item in duoyinzi:
    for word, pronunciations in item.items():
        duoyinzi_dict[word] = pronunciations

# 定义注音函数
def add_pronunciation(sentence):
    # jieba分词
    words = list(jieba.cut(sentence))
    output = []

    for word in words:
        found = False  # 确保是否找到多个音字
        for key, pronunciations in duoyinzi_dict.items():
            if word == key or (key in word):  # 检查匹配
                found = True
                word_temp = list(word)#把词变成列表
                # 检查当前词是否在对应的拼音词组中
                for pinyin_key, values in pronunciations.items():
                    if word in values:
                        pronunciation = pinyin_key  # 从对应的拼音选择
                        break
                else:
                    # 如果没有找到相应的词组，使用第一个拼音
                    pronunciation = list(pronunciations.keys())[0]
                
                for wo in word_temp:
                    if wo == key:
                        output.append(f"{wo}({pronunciation})")
                    else:
                        output.append(f"{wo}")
                    
                break  # 找到后停止继续查找

        if not found:
            output.append(word)  # 如果当前词不是多音字，则直接添加

    return ' '.join(output)

# 读取Word文档
try:
    doc = docx.Document(r'D:\Learning NLP Together\UCAS-NLP-Homework\text2.docx')
except FileNotFoundError:
    print("文件未找到，请检查文件路径和名称是否正确。")
    exit(1)

# 创建一个新的Word文档来存储结果
output_doc = docx.Document()
output_doc.add_paragraph("原始内容")

for para in doc.paragraphs:
    text = para.text.strip()  # 去除可能的前后空白
    #pinyin_text = ' '.join([char + '(' + ''.join(lazy_pinyin(char)) + ')' for char in text if '\u4e00' <= char <= '\u9fa5'])
    pinyin_text = add_pronunciation(text).replace(" ","")
    
    if text:  # 只添加非空的段落
        output_doc.add_paragraph(pinyin_text)

# 将结果保存到新的文件
output_doc.save('text2_with_pinyin.docx')

# V2.1 

In [1]:
import docx
from pypinyin import lazy_pinyin
import jieba

# 建立多音字列表
duoyinzi = [
    {"便": {
        "biàn": ["方便", "便利", "轻便", "方便面"],
        "pián": ["便宜", "便宜货"]
    }},
    {"好": {
        "hǎo": ["好人", "好吃"],
        "hào": ["爱好", "喜好"]
    }},
    {"打": {
        "dǎ": ["打拳", "打手"],
        "dá": ["一打"]
    }},
    {"行": {
        "háng": ["行业", "行走"],
        "xíng": ["同行", "行李"]
    }},
    {"重": {
        "zhòng": ["重量", "认真"],
        "chóng": ["重新", "重复"]
    }},
    {"乐": {
        "lè": ["快乐", "乐趣"],
        "yuè": ["音乐", "乐器"]
    }},
    {"时": {
        "shí": ["时间", "时刻"],
        "shì": ["时尚", "情况"]
    }},
    {"行": {
        "xíng": ["行走", "行李"],
        "háng": ["行业", "银行"]
    }},
    {"数": {
        "shǔ": ["数字", "数目"],
        "shù": ["数学", "数量"]
    }},
    {"难": {
        "nán": ["困难", "难做"],
        "nàn": ["灾难", "难堪"]
    }},
    {"长": {
        "cháng": ["长久", "成长"],
        "zhǎng": ["长官", "长辈"]
    }},
    {"正": {
        "zhèng": ["正确", "正义"],
        "zheng": ["正在"]
    }},
    {"下": {
        "xià": ["下面", "下来"],
        "xià": ["下降", "下班"]
    }},
    {"活": {
        "huó": ["生活", "活泼"],
        "huò": ["活着", "活人"]
    }},
    {"明": {
        "míng": ["明亮", "明白"],
        "mìng": ["命令", "命运"]
    }},
    {"取": {
        "qǔ": ["获取", "取舍"],
        "qǔ": ["取得", "取名"]
    }},
    {"如": {
        "rú": ["如果", "如意"],
        "rǔ": ["乳制品", "乳液"]
    }},
    {"乐": {
        "lè": ["快乐", "乐器"],
        "yuè": ["音乐", "乐部门"]
    }},
    {"提": {
        "tí": ["提问", "提取"],
        "tǐ": ["提携", "体力"]
    }},
    {"边": {
        "biān": ["边缘", "边界"],
        "piān": ["偏见", "偏北"]
    }},
    {"接": {
        "jiē": ["接受", "接待"],
        "jiē": ["接近", "接触"]
    }},
    {"错": {
        "cuò": ["错误", "错过"],
        "cuò": ["错位", "错账"]
    }},
    {"笑": {
        "xiào": ["笑声", "笑容"],
        "xiāo": ["笑话", "笑料"]
    }},
    {"示": {
        "shì": ["显示", "示范"],
        "shì": ["指示", "示威"]
    }},
    {"散": {
        "sàn": ["散步", "散开"],
        "sǎn": ["散发", "散文"]
    }},
    {"款": {
        "kuǎn": ["款式", "软件款"],
        "kuǎn": ["货款", "款待"]
    }},
    {"光": {
        "guāng": ["光明", "光荣"],
        "guāng": ["光线", "光波"]
    }},
    {"感": {
        "gǎn": ["感动", "感觉"],
        "gān": ["感情", "感冒"]
    }},
    {"头": {
        "tóu": ["头发", "头部"],
        "tóu": ["头疼", "头条"]
    }},
    {"种": {
        "zhǒng": ["种类", "种植"],
        "zhòng": ["种子", "种植园"]
    }},
    {"逼": {
        "bī": ["逼迫", "逼近"],
        "bì": ["威逼", "逼真"]
    }},
    {"会": {
        "huì": ["会议", "会见"],
        "kuài": ["会计", "会心"]
    }},
    {"计": {
        "jì": ["计划", "计算"],
        "jī": ["计谋", "计时"]
    }},
    {"破": {
        "pò": ["破碎", "破坏"],
        "pó": ["破土", "破茧"]
    }},
    {"胜": {
        "shèng": ["胜利", "胜任"],
        "shēng": ["生胜", "胜败"]
    }},
    {"频": {
        "pín": ["频繁", "频率"],
        "pìng": ["频密", "频发"]
    }},
    {"折": {
        "zhé": ["折扣", "折中"],
        "shé": ["折磨", "折服"]
    }},
    {"夺": {
        "duó": ["夺取", "夺走"],
        "dǔ": ["夺冠", "夺命"]
    }},
    {"器": {
        "qì": ["器官", "器械"],
        "qī": ["器量", "器重"]
    }},
    {"符": {
        "fú": ["符号", "符咒"],
        "pū": ["符文", "符号学"]
    }},
    {"随": {
        "suí": ["随便", "随意"],
        "zuì": ["随手", "随和"]
    }},
    {"脆": {
        "cuì": ["脆弱", "脆皮"],
        "cuì": ["脆响", "脆口"]
    }},
    {"减": {
        "jiǎn": ["减少", "减轻"],
        "jiàn": ["减肥", "减压"]
    }},
    {"坏": {
        "huài": ["坏处", "坏蛋"],
        "bài": ["坏话", "坏事"]
    }},
    {"滥": {
        "làn": ["滥用", "滥发"],
        "lǎn": ["滥情", "滥竽"]
    }},
    {"煮": {
        "zhǔ": ["煮饭", "煮水"],
        "zhù": ["煮蛋", "煮肉"]
    }},
    {"囚": {
        "qiú": ["囚犯", "囚禁"],
        "zhǔ": ["囚室", "囚徒"]
    }},
    {"点": {
        "diǎn": ["时间点", "点心"],
        "diàn": ["热点", "点击"]
    }},
    {"拦": {
        "lán": ["拦截", "拦路"],
        "làn": ["拦住", "拦网"]
    }},
    {"锤": {
        "chuí": ["锤子", "锤打"],
        "zhuī": ["锤炼", "锤击"]
    }},
    {"委": {
        "wěi": ["委屈", "委托"],
        "wēi": ["委任", "委屈"]
    }},
    {"满": {
        "mǎn": ["满意", "满分"],
        "màn": ["满载", "满腹"]
    }},
    {"模": {
        "mó": ["模型", "模范"],
        "mú": ["模糊", "模样"]
    }},
    {"击": {
        "jī": ["攻击", "击败"],
        "gī": ["击打", "击剑"]
    }},
    {"播": {
        "bō": ["播放", "播种"],
        "bà": ["播音", "播洒"]
    }},
    {"压": {
        "yā": ["压力", "压制"],
        "yà": ["压药", "压后"]
    }},
    {"追": {
        "zhuī": ["追赶", "追求"],
        "zhuǐ": ["追问", "追踪"]
    }},
    {"沽": {
        "gū": ["沽名", "沽酒"],
        "gōu": ["沽水", "沽货"]
    }},
    {"反": {
        "fǎn": ["反对", "反映"],
        "fàn": ["反复", "反响"]
    }},
    {"愈": {
        "yù": ["愈合", "愈快"],
        "yǔ": ["愈加", "愈演愈烈"]
    }},
    {"速": {
        "sù": ["速度", "迅速"],
        "sù": ["速记", "速战"]
    }},
    {"快": {
        "kuài": ["快乐", "快递"],
        "kuài": ["快餐", "快车"]
    }},
]

# 创建字典方便查询
duoyinzi_dict = {}
for item in duoyinzi:
    for word, pronunciations in item.items():
        duoyinzi_dict[word] = pronunciations

# 定义注音函数
def add_pronunciation(sentence):
    # jieba分词
    words = list(jieba.cut(sentence))
    output = []

    for word in words:
        found = False  # 确保是否找到多个音字
        for key, pronunciations in duoyinzi_dict.items():
            if word == key or (key in word):  # 检查匹配
                found = True
                word_temp = list(word)#把词变成列表
                # 检查当前词是否在对应的拼音词组中
                for pinyin_key, values in pronunciations.items():
                    if word in values:
                        pronunciation = pinyin_key  # 从对应的拼音选择
                        break
                else:
                    # 如果没有找到相应的词组，使用第一个拼音
                    pronunciation = list(pronunciations.keys())[0]
                
                for wo in word_temp:
                    if wo == key:
                        output.append(f"{wo}({pronunciation})")
                    else:
                        output.append(f"{wo}")
                    
                break  # 找到后停止继续查找

        if not found:
            output.append(word)  # 如果当前词不是多音字，则直接添加

    return ' '.join(output)

# 读取Word文档
try:
    doc = docx.Document(r'D:\Learning NLP Together\UCAS-NLP-Homework\text3.docx')
except FileNotFoundError:
    print("文件未找到，请检查文件路径和名称是否正确。")
    exit(1)

# 创建一个新的Word文档来存储结果
output_doc = docx.Document()
output_doc.add_paragraph("原始内容")

for para in doc.paragraphs:
    text = para.text.strip()  # 去除可能的前后空白
    #pinyin_text = ' '.join([char + '(' + ''.join(lazy_pinyin(char)) + ')' for char in text if '\u4e00' <= char <= '\u9fa5'])
    pinyin_text = add_pronunciation(text).replace(" ","")
    
    if text:  # 只添加非空的段落
        output_doc.add_paragraph(pinyin_text)

# 将结果保存到新的文件
output_doc.save('text3_with_pinyin.docx')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86185\AppData\Local\Temp\jieba.cache
Loading model cost 0.586 seconds.
Prefix dict has been built successfully.
